In [ ]:
#
# Valuation of American Options
# with Least-Squares Monte Carlo
# Primal and Dual Algorithm
# Case 1: American Put Option (APO)
# Case 2: Short Condor Spread (SCS)
#
import math 
import numpy as np
import pandas as pd
from time import time
from datetime import datetime
import itertools as it 
import warnings
warnings.simplefilter('ignore')


t0 = time()
np.random.seed(150000) # seed for Python RNG

## Simulation Parameters 
runs = 5
write = True
otype = [1, 2] # option type
M = [10, 20] # time steps
I1 = np.array([4, 6]) * 4096 # replication for regression
I2 = np.array([1, 2]) * 1024 # replication for valuation
J = [50, 75] # replication for nested MCS
reg = [5, 9] # no of basis functions
AP = [False, True] # antithetic paths
MM = [False, True] # moment matching of RN
ITM = [True, False] # ITM paths for regression 
results = pd.DataFrame()

In [ ]:
#
# Function Definitions
#

def generate_randomnumbers(I):
    ''' Function to generate I pseudo-random numbers.'''
    if AP:
        ran = np.random.standard_normal(I / 2)
        ran = np.concatenate((ran, -ran))
    else:
        ran = np.random.standard_normal(I)
    if MM:
        ran = ran - np.mean(ran)
        ran = ran / np.std(ran)
    return ran

def generate_paths(I):
    ''' Function to generate I stock paths. '''
    S = np.zeros((M + 1, I), dtype=np.float64) # stock matrix
    S[0] = S0 # initial values
    for t in range(1, M + 1, 1): # stock price paths
        ran = generate_random_numbers(I)
        S[t] = S[t - 1] * np.exp((r - sigma ** 2 / 2) * dt + sigma * ran * math.sqrt(dt))
    return S

def inner_values(S):
    ''' Inner value functions for American put and Short Condor Spread. '''
    if otype == 1:
        return np.maximum(40. - S, 0)
    else:
        return np.maximum(40., np.maximum(90. - S, 0) + np.maximum( - 110., 0))
    
